<a href="https://colab.research.google.com/github/MariaIsabelLL/ClasificacionTexto/blob/main/Clasificacion_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librería NLTK

In [ ]:
import nltk
#nltk.download('book')

#from nltk.book import *

#(text1)
#for i in range(0,50):
#  print(text1[i])
#print(len(text1))
#print(len(set(text1)))
#fdist = FreqDist(text1)
#fdist.plot(30, cumulative = False)


# Tokenización

In [ ]:
from nltk import word_tokenize

frase1 = 'Me gusta este webinar, es genial! Los webinar me gustan mucho.'
frase_tokenizada = word_tokenize(frase1)
print(frase1)
print(frase_tokenizada)


# Stopwords y Stemming

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

stopword_spanish = stopwords.words('spanish')
stopword_ingles = stopwords.words('english')
print(stopword_spanish)
print(stopword_ingles)

def removestopwords_ingles(texto):
  text_nuevo = [w.lower() for w in texto if w not in stopword_ingles and w not in string.punctuation]
  return text_nuevo

def removestopwords_espanol(texto):
  text_nuevo = [w.lower() for w in texto if w not in stopword_spanish and w not in string.punctuation]
  return text_nuevo

fdist2 = FreqDist(removestopwords_ingles(text1))
fdist2.plot(30, cumulative = False)

print(frase_tokenizada)
print(removestopwords_espanol(frase_tokenizada))
stemmer = SnowballStemmer('spanish')
stemmed_text = [stemmer.stem(i) for i in removestopwords_espanol(frase_tokenizada)]
print(stemmed_text)

# Datos: movie reviews

In [ ]:
from nltk.corpus import movie_reviews

print(len(movie_reviews.fileids()))
print(len(movie_reviews.categories()))

#freqDist = FreqDist(movie_reviews.words())
#freqDist.plot(30)

new_text = removestopwords_ingles(movie_reviews.words())
freqDist2 = FreqDist(new_text)
freqDist2.plot(30)
print(freqDist2.most_common(30))

# Procesamiento Datos: tokenize y BOW

In [ ]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
stemmer = PorterStemmer()

def remove_stemm(words):
    words_clean = []    
    for word in word_tokenize(words):
        word = word.lower()
        if word not in string.punctuation and word not in stopword_ingles:
            words_clean.append(stemmer.stem(word))
    return words_clean

def bag_of_words_CountVec(X1):    
    matrix_vectorizer = CountVectorizer(tokenizer=remove_stemm,analyzer='word',
                            max_features=1000)     
    X = matrix_vectorizer.fit_transform(X1).toarray()  
    return X,matrix_vectorizer

def DatosBOW():
    df = pd.DataFrame()
    X1 = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]
    y = [movie_reviews.categories(fileid)[0] for fileid in movie_reviews.fileids()]
    print(len(X1))
    print('Comentario 1',X1[0])
    print('Comentario 2',X1[1])
    
    (X,count_vectorizer) = bag_of_words_CountVec(X1)
    
    return X,y,count_vectorizer
  
(X,y,count_vectorizer) = DatosBOW()
print(X)
print(y)


def bag_of_words_CountVec2(X1,count_vectorizer):    
    X = count_vectorizer.transform(X1).toarray()  
    return X



# Clasificador Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def clasificadorBOW(): 
    (X,y,count_vectorizer) =  DatosBOW()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)
     
    mnb = MultinomialNB()
    classifier = mnb.fit(X_train, y_train)    
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)

    lista_texto = []
    lista_texto.append('This is an ugly movie.')
    lista_texto.append('This is an excelent movie.')
    lista_texto.append('This is an different movie.')
    prediccion = classifier.predict(bag_of_words_CountVec2(lista_texto,count_vectorizer)) 
    print(prediccion)

clasificadorBOW()  